# Solving the heat equation with Crank-Nicolson

[AMath 586, Spring Quarter 2016](http://faculty.washington.edu/rjl/classes/am586s2016/)

Sample program to solve the heat equation with the Crank-Nicolson method.


In [1]:
%pylab inline

The function below solves the heat equation $u_t = \kappa u_{xx}$ on the interval $0\leq x \leq 1$ with Dirichlet boundary conditions $u(0,t) = g_0(t)$ and $u(1,t) = g_1(t)$.  

It is set up to solve the equation for a case where the exact solution is known, the decaying Gaussian
$$
u(x,t) = \frac{1}{\sqrt{4\beta\kappa t + 1}} \exp\left(\frac{-(x-x_0)^2}{4\kappa t + 1/\beta}\right).
$$
The initial data and boundary conditions are obtained by evaluating this function at $t=0$ or at $x=0$ or $x=1$. In particular, the initial conditions are simply
$$
u(x,0) = \eta(x) = \exp(-\beta(x-x_0)^2).
$$


In [2]:

def heat_CN(m,nsteps,nplot=None):
    """
    Solve u_t = kappa * u_{xx} on [ax,bx] with Dirichlet boundary conditions,
    using the Crank-Nicolson method with m interior points, taking nsteps
    time steps.  
    
    If nplot is not None, a plot will be generated and the max-norm of error
    printed every nplot time steps and at the final time.
    
    Returns dt, h, and the max-norm of the error.
    This routine can be embedded in a loop on m to test the accuracy.
    
    Note: the vector x defined below is of length m+2 and includes both boundary points.
    The vector uint is of length m and is only the interior points that we solve for,
    by solving an m by m linear system each time step.
    The vector u is of length m+2 and obtained by extending uint with the boundary values,
    so that plotting (x,u) includes the boundary values.
    
    """

    from scipy import sparse
    from scipy.sparse.linalg import spsolve
        
    ax = 0
    bx = 1
    kappa = .02               # heat conduction coefficient:
    tfinal = 1.               # final time
    
    h = (bx-ax)/float(m+1)    # h = delta x
    x = linspace(ax,bx,m+2)   # note x(1)=0 and x(m+2)=1
                               # u(1)=g0 and u(m+2)=g1 are known from BC's
    dt = tfinal / float(nsteps)
    
    
    # true solution for comparison:
    # For Gaussian initial conditions u(x,0) = exp(-beta * (x-x0)^2)
    beta = 150
    x0 = 0.4
    utrue = lambda x,t: exp(-(x-0.4)**2 / (4*kappa*t + 1./beta)) \
                / sqrt(4*beta*kappa*t+1.) 
    
    # initial conditions:
    u0 = utrue(x,0)
    
    
    # Each time step we solve MOL system U' = AU + g using the Trapezoidal method
    
    # set up matrices:
    r = 0.5 * kappa* dt/(h**2)
    em = ones(m)
    em1 = ones(m-1)
    A = sparse.diags([em1, -2*em, em1], [-1, 0, 1], shape=(m,m))
    A1 = sparse.eye(m) - r * A
    A2 = sparse.eye(m) + r * A
    
    
    # initial data on fine grid for plotting:
    xfine = linspace(ax,bx,1001)
    ufine = utrue(xfine,0)
    
    # initialize u and plot:
    tn = 0
    u = u0
    
    if nplot is not None:
        print "h = %g, dt = %g"  % (h,dt)
        plot(x,u,'b.-',label='computed')
        plot(xfine,ufine,'r',label='true')
        legend()
        ylim([-0.1, 1.1])
        title('Plot every %i timesteps with m = %i, nsteps = %i' % (nplot,m,nsteps))    
    
    
    # main time-stepping loop:
    
    for n in range(1,nsteps+1):
        tnp = tn + dt   # = t_{n+1}
    
        # boundary values u(0,t) and u(1,t) at times tn and tnp:
        # already set at time tn in array u:
        g0n = u[0]
        g1n = u[m+1]
        
        # evaluate true solution to get new boundary values at tnp:
        g0np = utrue(ax,tnp)
        g1np = utrue(bx,tnp)
    
        # compute right hand side for linear system:
        uint = u[1:m+1]   # interior points (unknowns)
        rhs = A2.dot(uint)   # sparse matrix-vector product A2 * uint
        # fix-up right hand side using BC's (i.e. add vector g to A2*uint)
        rhs[0] = rhs[0] + r*(g0n + g0np)
        rhs[m-1] = rhs[m-1] + r*(g1n + g1np)
    
        # solve linear system:
        uint = spsolve(A1,rhs)   # sparse solver
            
        # augment with boundary values:
        u = hstack([g0np, uint, g1np])
        
        error = abs(u-utrue(x,tnp)).max()
    
        # plot results at desired times:
        if nplot is not None:
            if mod(n,nplot)==0 or n==nsteps:
                ufine = utrue(xfine,tnp)
                plot(x,u,'b.-',label='computed')
                plot(xfine,ufine,'r',label='true')                
                print 'at time t = %9.5e  max error =  %9.5e' % (tnp,error)
    
        tn = tnp   # for next time step
    
    return h,dt,error 

## Test this with a few values of m and nsteps:

In [3]:
h,dt,error = heat_CN(9,10,2)

In [4]:
h,dt,error = heat_CN(19,20,4)

## Test for second-order accuracy

If dt and h are both reduced by 2, the error should go down by a factor 4 (for sufficiently small values).

In [5]:
nsteps_vals = [20,40,80,160,320]  # values to test
E = empty(len(nsteps_vals))

# print table header:
print "   h         dt          error      ratio  estimated order"

for j,nsteps in enumerate(nsteps_vals):
    h,dt,E[j] = heat_CN(m=nsteps-1, nsteps=nsteps)
    if j>0:
        ratio = E[j-1] / E[j]
    else:
        ratio = nan
    p = log(ratio)/log(2)
    print "%8.6f  %8.6f  %12.8f    %4.2f        %4.2f" % (h, dt, E[j], ratio, p)

loglog(nsteps_vals, E, '-o')
title('Log-log plot of errors')
xlabel('nsteps')
ylabel('error')

## Observe oscillations if dt is too large

We know that Crank-Nicolson is stable for any time step, but the amplification factor approaches $-1$ as $k\lambda \rightarrow \infty$, so we expect high wavenumber modes to oscillate in time if we take the time step too large. This can be observed with the Gaussian initial data used here.

In [6]:
h,dt,error = heat_CN(39,2,1)